In [1]:
import pandas as pd
import numpy as np
from dataframe import *

print(features_df.iloc[40])


<bound method NDFrame.head of         Vmax      Vmin    Vrange  rest_voltage_after_discharge  \
0   4.200795  1.999667  2.201128                      2.847339   
1   4.200795  1.999362  2.201433                      2.839618   
2   4.200795  1.999057  2.201739                      2.833239   
3   4.200795  1.999972  2.200823                      2.828356   
4   4.200795  2.864309  1.336486                           NaN   
..       ...       ...       ...                           ...   
85  4.201031  1.998680  2.202351                      2.738758   
86  4.201031  1.998070  2.202961                      2.740283   
87  4.201031  1.997459  2.203571                      2.741870   
88  4.201031  1.998680  2.202351                      2.743793   
89  4.200726  2.829025  1.371701                           NaN   

    dvdt_discharge chemistry  
0        -0.001893       LCO  
1        -0.001929       LCO  
2        -0.001965       LCO  
3        -0.001946       LCO  
4              NaN    

In [ ]:

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

X = features_df.drop(columns=["chemistry"])  
y = features_df["chemistry"]


imputer = SimpleImputer(strategy="mean")
X_imputed = imputer.fit_transform(X)


X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=56)


rf = RandomForestClassifier(n_estimators=100, random_state=9)
rf.fit(X_train, y_train)


y_pred = rf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

y_pred = rf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
importance_df = pd.DataFrame({
    "feature": X.columns,
    "importance": rf.feature_importances_
}).sort_values(by="importance", ascending=False)

print(importance_df)



Accuracy: 1.0
Confusion Matrix:
 [[6 0 0]
 [0 6 0]
 [0 0 6]]
Classification Report:
               precision    recall  f1-score   support

         LCO       1.00      1.00      1.00         6
         LFP       1.00      1.00      1.00         6
         NMC       1.00      1.00      1.00         6

    accuracy                           1.00        18
   macro avg       1.00      1.00      1.00        18
weighted avg       1.00      1.00      1.00        18

Accuracy: 1.0
                        feature  importance
0                          Vmax    0.307022
3  rest_voltage_after_discharge    0.266803
4                dvdt_discharge    0.177639
1                          Vmin    0.130525
2                        Vrange    0.118010
